In [ ]:
import pandas as pd

df_clinical = pd.read_excel('prad_tcga_clinical_data.xlsx')
df_genes = pd.read_excel('prad_tcga_genes.xlsx')

In [ ]:
df_clinical = df_clinical[['PATIENT_ID', 'GLEASON_PATTERN_PRIMARY', 'GLEASON_PATTERN_SECONDARY', 'GLEASON_SCORE']]

In [ ]:
# Encoding...
# New_Gleason_Score -> Gleason_Score_Primary + Gleason_Score_Secondary
# 1 -> 3 + 3
# 2 -> 3 + 4
# 3 -> 4 + 3
# 4 -> 4 + 4
# 5 -> 4 + 5 or 5 + 4 or 5 + 5
df_clinical.loc[df_clinical['GLEASON_SCORE'] == 6, 'GLEASON_SCORE'] = 1
df_clinical.loc[(df_clinical['GLEASON_PATTERN_PRIMARY'] == 3) & (df_clinical['GLEASON_PATTERN_SECONDARY'] == 4), 'GLEASON_SCORE'] = 2
df_clinical.loc[(df_clinical['GLEASON_PATTERN_PRIMARY'] == 4) & (df_clinical['GLEASON_PATTERN_SECONDARY'] == 3), 'GLEASON_SCORE'] = 3
df_clinical.loc[df_clinical['GLEASON_SCORE'] == 8, 'GLEASON_SCORE'] = 4
df_clinical.loc[df_clinical['GLEASON_SCORE'] >= 9, 'GLEASON_SCORE'] = 5

In [ ]:
df_clinical['GLEASON_SCORE'].value_counts()

2    145
5    141
3    101
4     63
1     44
Name: GLEASON_SCORE, dtype: int64

In [ ]:
df_clinical

,PATIENT_ID,GLEASON_PATTERN_PRIMARY,GLEASON_PATTERN_SECONDARY,GLEASON_SCORE
0,TCGA-2A-A8VO,3,3,1
1,TCGA-2A-A8VT,4,5,5
2,TCGA-2A-A8VV,3,3,1
3,TCGA-2A-A8VX,4,4,4
4,TCGA-2A-A8W1,4,3,3
...,...,...,...,...
494,TCGA-ZG-A9M4,5,4,5
495,TCGA-ZG-A9MC,5,4,5
496,TCGA-ZG-A9N3,5,4,5
497,TCGA-ZG-A9ND,5,4,5


In [ ]:
df_genes_trans = df_genes.transpose()
columns = df_genes_trans.iloc[0][:-6]

# We drop the patients whose gene information is not present...
df_genes_trans.drop([60483, 60484, 60485, 60486, 60487, 60488], axis=1, inplace=True)
df_genes_trans.columns = columns
df_genes_trans.drop(['ID'], axis=0, inplace=True)
df_genes_trans['PATIENT_ID'] = df_genes_trans.index
df_genes_trans = df_genes_trans.reset_index()
df_genes_trans.drop(['index'], axis=1, inplace=True)

In [ ]:
selected_patients = set(df_clinical['PATIENT_ID']).intersection(set(df_genes_trans['PATIENT_ID']))

In [ ]:
df_clinical = df_clinical[df_clinical['PATIENT_ID'].isin(selected_patients)]

In [ ]:
df_clinical = df_clinical.reset_index()
df_clinical.drop(['index'], axis=1, inplace=True)

In [ ]:
df_merged = pd.merge(df_clinical, df_genes_trans, on="PATIENT_ID")

In [ ]:
df_merged

,PATIENT_ID,GLEASON_PATTERN_PRIMARY,GLEASON_PATTERN_SECONDARY,GLEASON_SCORE,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
0,TCGA-2A-A8VO,3,3,1,0,0.011071,3.906517,0,1.780208,7.896705,...,0,2.866661,0,0.256907,0,7.607006,0.485376,0,0.375844,0
1,TCGA-2A-A8VT,4,5,5,0.027121,0.007526,2.62162,0,0.885415,16.211265,...,0,0.0429,0,0.043665,0.045202,16.448059,0.329985,0,0.432963,0
2,TCGA-2A-A8VV,3,3,1,0,0,2.502227,0,1.00738,12.855196,...,0,0.032524,0,0.231727,0,8.758392,0.395435,0,0.210937,0
3,TCGA-2A-A8VX,4,4,4,0,0.016101,5.116545,0,1.001573,11.957889,...,0,6.594702,0,0.09341,0,7.768841,0.330189,0,0.129063,0
4,TCGA-2A-A8W1,4,3,3,0,0,1.928498,0,1.931438,9.737653,...,0,0.108911,0,0.048497,0,8.996123,0.047291,0,0.204965,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,TCGA-ZG-A9M4,5,4,5,0,0.007774,2.980603,0,1.085584,11.631324,...,0,0.196234,0,0.090201,0.046688,9.00045,0.26387,0,0.659797,0
490,TCGA-ZG-A9MC,5,4,5,0,0.009559,2.975541,0,4.26352,14.747407,...,0,0.054486,0,0.221828,0.05741,15.506879,0.135194,0,0.937517,0
491,TCGA-ZG-A9N3,5,4,5,0,0,2.650948,0,4.084599,7.349847,...,0,0.015595,0,0.222218,0,10.874603,0.297948,0,0.288973,0
492,TCGA-ZG-A9ND,5,4,5,0,0,3.121445,0,2.844586,7.001207,...,0,0.071485,0,0,0,9.572822,0.159634,0,0.130096,0


In [ ]:
df_merged.to_pickle('df_merged_gs_updated.pkl')